In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Fri Feb 14 14:16:04 2020

@author: thomas
"""

#MODULES
import os,sys
import re
import numpy as np
import pandas as pd
from mpl_toolkits import mplot3d
%matplotlib notebook
import matplotlib as mpl
#mpl.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.ticker import (MultipleLocator,AutoMinorLocator)
from scipy.signal import savgol_filter
import pathlib
from matplotlib import animation
from IPython.display import display, Image, HTML

mpl.rcParams['axes.linewidth'] = 1.5 #set the value globally

In [2]:
#CONSTANTS
cwd_PYTHON = os.getcwd()
PERIOD = 0.1
DT = 5.0e-3
RADIUSLARGE = 0.002
RADIUSSMALL = 0.001
csfont = {'fontname':'Times New Roman'}

#Lists
#RLength
ThetaList=['0.0','22.5','45.0','67.5','90.0','112.5','135.0','157.5','180.0',
           '202.5','225.0','247.5','270.0','292.5','315.0','337.5']
HxList=['-0.5','0.5','1.5','2.5','3.5','4.5','5.5','6.5','7.5','8.5','9.5','10.5','11.5','12.5']
HyList=['-13','-11','-9','-7','-5','-3','-1','1','3','5','7','9']

allData = []

In [3]:
import matplotlib
def WhichEndState(data,scheme):
    #In this function, we will determine if the pair are in an end state configuration
    #Diverge = 0: Red
    #V-Shape = 1: Blue
    #Orbit = 2: pink
    #NOTA = 3: black
    
    #Diverge: H_bw > 10
    #V-Shape: Theta = 29 or 331; H_bw ~ 3.0R
    #Headbutt: Theta = 180; H_bw = 3.2-3.4R
    #Orbit: Theta = 180; H_bw = 3.0-3.25R
    
    #data = data.sort_values(by=['parHx','parHy','time'])
    data = data.reset_index(drop=True)
    data['State'] = 4
    data['color'] = 'black'
    data['Theta_deg'] = 180.0*data['Theta']
    data['color_r'] = 0
    data['color_g'] = 0
    data['color_b'] = 0
    color_value = matplotlib.colors.to_rgb('black')
    for idx in range(len(data['time'])):
        #Which End State?
        #Diverge
        if(data.loc[idx,'H'] > 10.0):
            data.loc[idx,'State'] = 0
            data.loc[idx,'color'] = 'gray'
            color_value = matplotlib.colors.to_rgb('gray')
            data.loc[idx,'color_r'] = color_value[0]
            data.loc[idx,'color_g'] = color_value[1]
            data.loc[idx,'color_b'] = color_value[2]
        #V-Shape
        elif((data.loc[idx,'H'] < 3.1 and data.loc[idx,'H'] >= 2.8) and 
             ((data.loc[idx,'Theta_deg'] < 32.0 and data.loc[idx,'Theta_deg'] >= 26.0) or
              (data.loc[idx,'Theta_deg'] < 334.0 and data.loc[idx,'Theta_deg'] >= 328.0))):
            data.loc[idx,'State'] = 1
            data.loc[idx,'color'] = 'blue'
            color_value = matplotlib.colors.to_rgb('blue')
            if(scheme=='bright'):
                data.loc[idx,'color_r'] = 141
                data.loc[idx,'color_g'] = 160
                data.loc[idx,'color_b'] = 203
            elif(scheme=='dark'):
                data.loc[idx,'color_r'] = 117
                data.loc[idx,'color_g'] = 112
                data.loc[idx,'color_b'] = 179
        #Orbit
        elif((data.loc[idx,'H'] < 3.25 and data.loc[idx,'H'] >= 3.0) and 
             (data.loc[idx,'Theta_deg'] < 185.0 and data.loc[idx,'Theta_deg'] >= 175.0) ):
            data.loc[idx,'State'] = 2
            data.loc[idx,'color'] = 'pink'
            color_value = matplotlib.colors.to_rgb('pink')
            if(scheme=='bright'):
                data.loc[idx,'color_r'] = 231
                data.loc[idx,'color_g'] = 138
                data.loc[idx,'color_b'] = 195
            elif(scheme=='dark'):
                data.loc[idx,'color_r'] = 231
                data.loc[idx,'color_g'] = 41
                data.loc[idx,'color_b'] = 138
        #Headbutt
        elif((data.loc[idx,'H'] < 3.5 and data.loc[idx,'H'] >= 3.25) and 
             (data.loc[idx,'Theta_deg'] < 185.0 and data.loc[idx,'Theta_deg'] >= 175.0) and
             (data.loc[idx,'Hx'] <= 0.1 and data.loc[idx,'Hx'] > -0.1)):
            data.loc[idx,'State'] = 3
            data.loc[idx,'color'] = 'orange'
            color_value = matplotlib.colors.to_rgb('orange')
            if(scheme=='bright'):
                data.loc[idx,'color_r'] = 252
                data.loc[idx,'color_g'] = 131
                data.loc[idx,'color_b'] = 98
            elif(scheme=='dark'):
                data.loc[idx,'color_r'] = 217
                data.loc[idx,'color_g'] = 95
                data.loc[idx,'color_b'] = 2
        else:
            color_value= matplotlib.colors.to_rgb('black')
        #data.loc[idx,'color_r'] = color_value[0]
        #data.loc[idx,'color_g'] = color_value[1]
        #data.loc[idx,'color_b'] = color_value[2]
    
    return data

In [6]:
import plotly.graph_objects as go

#Grab Last timestep configuration data and combine in new dataset
#Using new dataset, plot in 3D the phase space (Hx, Hy, Theta)
#Maybe plotly, maybe matplotlib
#Plot Torus/Donut
#Make surface

allData = pd.read_csv(cwd_PYTHON+'/allData_Re10.csv',delimiter=' ')

endData = allData[allData['time'] == 20.0].copy()
endData = endData[endData['parHx'] >= 0.0].copy()
endData = endData.sort_values(by=['parTheta','parHx','parHy'])
endData = endData.reset_index(drop=True)
endData['xval'] = 0.5*endData['parHx']*np.cos(endData['parTheta']*np.pi/180.0)
endData['yval'] = 0.5*endData['parHx']*np.sin(endData['parTheta']*np.pi/180.0)
endData['zval'] = 0.5*endData['parHy']
endData = endData.sort_values(by=['zval','yval','xval'])
endData = endData.reset_index(drop=True)
scheme='dark'
endData = WhichEndState(endData,scheme)

endData['ColorString'] = 0.0
for idx in range(len(endData['color_r'])):
    endData.loc[idx,'ColorString'] = 'rgb({0},{1},{2})'.format(endData.loc[idx,'color_r'],endData.loc[idx,'color_g'],endData.loc[idx,'color_b'])

#Diverge = 0: Red
#V-Shape = 1: Blue
#Orbit = 2: pink
#NOTA = 3: black
    
#Create traces for each end state
traces = [0 for a in range(7)]
opaque = 0.4

#Divergence
divergeData = endData[endData['State'] == 0].copy()
divergeData = divergeData.reset_index(drop=True)

div1Data = divergeData[divergeData['zval'] >= 0.0].copy()
div1Data = div1Data.reset_index(drop=True)
div2Data = divergeData[divergeData['zval'] < 0.0].copy()
div2Data = div2Data.reset_index(drop=True)
div3Data = div2Data[div2Data['xval'] >= 0.0].copy()
div3Data = div3Data.reset_index(drop=True)
div4Data = div2Data[div2Data['xval'] < 0.0].copy()
div4Data = div4Data.reset_index(drop=True)

print(div1Data.loc[0,'ColorString'])

traces[0] = go.Mesh3d(x=div1Data['xval'], 
                     y=div1Data['yval'], 
                     z=div1Data['zval'], 
                     #color=div1Data.loc[0,'color'],
                     color=div1Data.loc[0,'ColorString'],
                     opacity=opaque,
                     #opacity=1.0,
                     alphahull=2.25
                     )

traces[1] = go.Mesh3d(x=div3Data['xval'], 
                     y=div3Data['yval'], 
                     z=div3Data['zval'], 
                     #color=div3Data.loc[0,'color'], 
                     color=div3Data.loc[0,'ColorString'],
                     opacity=opaque,
                     #opacity=1.0,
                     alphahull=1.61
                     )

traces[2] = go.Mesh3d(x=div4Data['xval'], 
                     y=div4Data['yval'], 
                     z=div4Data['zval'], 
                     #color=div4Data.loc[0,'color'], 
                     color=div4Data.loc[0,'ColorString'],
                     opacity=opaque,
                     #opacity=1.0,
                     alphahull=3
                     )

#V-Shape
VshapeData = endData[endData['State'] == 1].copy()
VshapeData = VshapeData.reset_index(drop=True)
traces[3] = go.Mesh3d(x=VshapeData['xval'], 
                     y=VshapeData['yval'], 
                     z=VshapeData['zval'], 
                     #color=VshapeData.loc[0,'color'], 
                     color=VshapeData.loc[0,'ColorString'],
                     opacity=opaque,
                     #opacity=1.0,
                     alphahull=8
                     )

#Orbit
orbitData = endData[endData['State'] == 2].copy()
orbitData = orbitData.reset_index(drop=True)
traces[4] = go.Mesh3d(x=orbitData['xval'], 
                     y=orbitData['yval'], 
                     z=orbitData['zval'], 
                     #color=orbitData.loc[0,'color'], 
                     color=orbitData.loc[0,'ColorString'],
                     opacity=opaque,
                     #opacity=1.0,
                     alphahull=8
                     )

#Headbutt
headbuttData = endData[endData['State'] == 3].copy()
headbuttData = headbuttData.reset_index(drop=True)
traces[5] = go.Mesh3d(x=headbuttData['xval'], 
                     y=headbuttData['yval'], 
                     z=headbuttData['zval'], 
                     #color=headbuttData.loc[0,'color'], 
                     color=headbuttData.loc[0,'ColorString'],
                     opacity=opaque,
                     #opacity=1.0,
                     alphahull=5
                     )

traces[6] = go.Scatter3d(
    x=endData['xval'],
    y=endData['yval'],
    z=endData['zval'],
    mode='markers',
    #marker_symbol='square',
    marker=dict(
        size=12,
        #color=endData['color'],                # set color to an array/list of desired values
        color=endData['ColorString'],
        #colorscale='Viridis',   # choose a colorscale
        opacity=1.0,
        line=dict(
                color='Black',
                width=2)
    )
)

layout = go.Layout(
    #title='Re10: 3D Phase Diagram',
    scene=dict(
        xaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=False,
            backgroundcolor='rgb(230, 230,230)'
        ),
        yaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=False,
            backgroundcolor='rgb(230, 230,230)'
        ),
        zaxis=dict(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=False,
            backgroundcolor='rgb(230, 230,230)'
        )
    ),
    showlegend=False,
)

fig = go.Figure(data = traces, layout=layout)

# tight layout
fig.update_layout(autosize=False,margin=dict(l=0, r=0, b=0, t=0))
fig.update_layout(scene=dict(
                    #xaxis_title=r'Hx*cos(theta) (R)',
                    #yaxis_title=r'Hx*sin(theta) (R)',
                    #zaxis_title=r'Hy (R)'),
                    #xaxis_title='H<sub>x<\sub>cos(&theta) (R)',
                    #yaxis_title='H<sub>x<\sub>sin(&theta) (R)',
                    #zaxis_title='H<sub>y<\sub> (R)',
                    xaxis_title=dict(text="H<sub>x</sub> cos("+u"\u03B8"+") (R)",font=dict(size=18,family='Times New Roman')),
                    yaxis_title=dict(text="H<sub>x</sub> sin("+u"\u03B8"+") (R)",font=dict(size=18,family='Times New Roman')),
                    zaxis_title=dict(text="H<sub>y</sub> (R)",font=dict(size=18,family='Times New Roman')),
                    ),
                    width=2000,height=2000,
                    margin=dict(r=100, b=100, l=100, t=100),
                    font=dict(
                            family="Times New Roman",
                            size=12,
                            color="black"
                            )
            )
fig.update_xaxes(automargin=True)
fig.update_yaxes(automargin=True)
#fig.update_zaxes(automargin=True)

camera = dict(
    up=dict(x=0.0, y=1.0, z=0.0),
    eye=dict(x=0., y=0., z=2.0)
)

fig.update_layout(scene_camera=camera)
#fig.show()

if not os.path.exists(cwd_PYTHON+'/../Figures/3D_Diagram'):
    os.mkdir(cwd_PYTHON+'/../Figures/3D_Diagram')
fig.write_image(cwd_PYTHON+'/../Figures/3D_Diagram/LLT_Re10_topview.png')
print('Done Making Top View')
camera = dict(
    up=dict(x=0.0, y=1.0, z=0.0),
    eye=dict(x=0., y=0., z=-2.0)
)

fig.update_layout(scene_camera=camera)
#fig.show()
fig.write_image(cwd_PYTHON+'/../Figures/3D_Diagram/LLT_Re10_botview.png')
print('Done Making Bot View')
camera = dict(
    up=dict(x=0.0, y=0.0, z=1.0),
    eye=dict(x=0.0, y=2.0, z=0.0)
)

fig.update_layout(scene_camera=camera)
#fig.show()
fig.write_image(cwd_PYTHON+'/../Figures/3D_Diagram/LLT_Re10_yposview.png')
print('Done Making Ypos View')
camera = dict(
    up=dict(x=0.0, y=0.0, z=1.0),
    eye=dict(x=0.0, y=-2.0, z=0.0),
    #center=dict(x=0, y=0, z=1.0)
)

fig.update_layout(scene_camera=camera)
#fig.show()
fig.write_image(cwd_PYTHON+'/../Figures/3D_Diagram/LLT_Re10_ynegview.png')
print('Done Making Yneg View')



rgb(0.5019607843137255,0.5019607843137255,0.5019607843137255)
Done Making Top View
Done Making Bot View
Done Making Ypos View
Done Making Yneg View


In [11]:
import plotly.graph_objects as go

#Grab Last timestep configuration data and combine in new dataset
#Using new dataset, plot in 3D the phase space (Hx, Hy, Theta)
#Maybe plotly, maybe matplotlib
#Plot Torus/Donut
#Make surface

allData = pd.read_csv(cwd_PYTHON+'/allData_Re10.csv',delimiter=' ')

endData = allData[allData['time'] == 20.0].copy()
endData = endData[endData['parHx'] >= 0.0].copy()
#endData = endData[endData['parHy'] <= 8.2].copy()
#endData = endData.sort_values(by=['parTheta','parHx','parHy'])
endData = endData.sort_values(by=['parHy','parTheta','parHx'])
endData['xval'] = 0.5*endData['parHx']*np.cos(endData['parTheta']*np.pi/180.0)
endData['yval'] = 0.5*endData['parHx']*np.sin(endData['parTheta']*np.pi/180.0)
endData['zval'] = 0.5*endData['parHy']
endData = endData.sort_values(by=['zval','yval','xval'])
endData = endData.reset_index(drop=True)
endData = WhichEndState(endData)

#Diverge = 0: Red
#V-Shape = 1: Blue
#Orbit = 2: pink
#NOTA = 3: black
    
#Create traces for each end state
traces = [0 for a in range(3)]
opaque = 0.2

#V-Shape
VshapeData = endData[endData['State'] == 1].copy()
VshapeData = VshapeData.reset_index(drop=True)

traces[0] = go.Mesh3d(x=VshapeData['xval'], 
                     y=VshapeData['yval'], 
                     z=VshapeData['zval'], 
                     color=VshapeData.loc[0,'color'], 
                     opacity=opaque,
                     #opacity=1.0,
                     alphahull=8
                     )

traces[1] = go.Scatter3d(
    x=VshapeData['xval'],
    y=VshapeData['yval'],
    z=VshapeData['zval'],
    mode='markers',
    #marker_symbol='square',
    marker=dict(
        size=4,
        color=VshapeData['color'],                # set color to an array/list of desired values
        #colorscale='Viridis',   # choose a colorscale
        opacity=1.0,
        line=dict(
                color='black',
                width=2)
    ),
    line=dict(
              color='black',
              width=2)
)

NOTAData = endData[endData['State'] == 4].copy()
NOTAData = NOTAData.reset_index(drop=True)

traces[2] = go.Scatter3d(
    x=NOTAData['xval'],
    y=NOTAData['yval'],
    z=NOTAData['zval'],
    mode='markers',
    #marker_symbol='square',
    marker=dict(
        size=4,
        color=NOTAData['color'],                # set color to an array/list of desired values
        #colorscale='Viridis',   # choose a colorscale
        opacity=1.0,
        line=dict(
                color='Black',
                width=2)
    )
)

fig = go.Figure(data = traces)

# tight layout
fig.update_layout(autosize=False,margin=dict(l=0, r=0, b=0, t=0))
fig.update_layout(scene = dict(
                    xaxis_title=r'H<sub>x<\sub>cos(&theta) (R)',
                    yaxis_title=r'H<sub>x<\sub>sin(&theta) (R)',
                    zaxis_title=r'H<sub>y<\sub> (R)'),
                    width=1000,height=1000,
                    margin=dict(r=20, b=10, l=10, t=10),
                    font=dict(
                            family="Times New Roman",
                            size=18,
                            color="black"
    ))
#fig.update_layout(showlegend=True,
#                  legend_title_text='Configurations\nRed=Diverge\nBlue=V-shape\nPink=Orbit\nOrange=Headbutt',
#                  font=dict(color="black",size=16))
camera = dict(
    eye=dict(x=0., y=0., z=2.5)
)

fig.update_layout(scene_camera=camera)
fig.show()

'''
#Export V-Shape Data as a point cloud

Vshape_pc_dict = {'x':VshapeData['xval'].values.tolist(),'y':VshapeData['yval'].values.tolist(),
                  'z':VshapeData['zval'].values.tolist(),'r':VshapeData['color_r'].values.tolist(),
                  'g':VshapeData['color_g'].values.tolist(),'b':VshapeData['color_b'].values.tolist()}
Vshape_pc_df = pd.DataFrame(data=Vshape_pc_dict)
Vshape_pc_df.to_csv(cwd_PYTHON+'/Vshape_pointcloud.csv',sep=' ',float_format='%.5e',index=False)
'''


"\n#Export V-Shape Data as a point cloud\n\nVshape_pc_dict = {'x':VshapeData['xval'].values.tolist(),'y':VshapeData['yval'].values.tolist(),\n                  'z':VshapeData['zval'].values.tolist(),'r':VshapeData['color_r'].values.tolist(),\n                  'g':VshapeData['color_g'].values.tolist(),'b':VshapeData['color_b'].values.tolist()}\nVshape_pc_df = pd.DataFrame(data=Vshape_pc_dict)\nVshape_pc_df.to_csv(cwd_PYTHON+'/Vshape_pointcloud.csv',sep=' ',float_format='%.5e',index=False)\n"

In [39]:
import plotly.graph_objects as go

#Grab Last timestep configuration data and combine in new dataset
#Using new dataset, plot in 3D the phase space (Hx, Hy, Theta)
#Maybe plotly, maybe matplotlib
#Plot Torus/Donut
#Make surface

allData = pd.read_csv(cwd_PYTHON+'/allData_Re10.csv',delimiter=' ')

endData = allData[allData['time'] == 20.0].copy()
endData = endData[endData['parHx'] >= 0.0].copy()
#endData = endData[endData['parHy'] <= 8.2].copy()
#endData = endData.sort_values(by=['parTheta','parHx','parHy'])
endData = endData.sort_values(by=['parHy','parTheta','parHx'])
endData['xval'] = 0.5*endData['parHx']*np.cos(endData['parTheta']*np.pi/180.0)
endData['yval'] = 0.5*endData['parHx']*np.sin(endData['parTheta']*np.pi/180.0)
endData['zval'] = 0.5*endData['parHy']
endData = endData.sort_values(by=['zval','yval','xval'])
endData = endData.reset_index(drop=True)
endData = WhichEndState(endData)

#Diverge = 0: Red
#V-Shape = 1: Blue
#Orbit = 2: pink
#NOTA = 3: black
    
#Create traces for each end state
traces = [0 for a in range(4)]
opaque = 0.4

#Divergence
divergeData = endData[endData['State'] == 0].copy()
divergeData = divergeData.reset_index(drop=True)

div1Data = divergeData[divergeData['zval'] >= 0.0].copy()
div1Data = div1Data.reset_index(drop=True)
div2Data = divergeData[divergeData['zval'] < 0.0].copy()
div2Data = div2Data.reset_index(drop=True)
div3Data = div2Data[div2Data['xval'] >= 0.0].copy()
div3Data = div3Data.reset_index(drop=True)
div4Data = div2Data[div2Data['xval'] < 0.0].copy()
div4Data = div4Data.reset_index(drop=True)

traces[0] = go.Mesh3d(x=div1Data['xval'], 
                     y=div1Data['yval'], 
                     z=div1Data['zval'], 
                     color=div1Data.loc[0,'color'], 
                     opacity=opaque,
                     #opacity=1.0,
                     alphahull=2.25
                     )

traces[1] = go.Mesh3d(x=div3Data['xval'], 
                     y=div3Data['yval'], 
                     z=div3Data['zval'], 
                     color=div3Data.loc[0,'color'], 
                     opacity=opaque,
                     #opacity=1.0,
                     alphahull=1.5
                     )

traces[2] = go.Mesh3d(x=div4Data['xval'], 
                     y=div4Data['yval'], 
                     z=div4Data['zval'], 
                     color=div4Data.loc[0,'color'], 
                     opacity=opaque,
                     #opacity=1.0,
                     alphahull=3
                     )

traces[3] = go.Scatter3d(
    x=divergeData['xval'],
    y=divergeData['yval'],
    z=divergeData['zval'],
    mode='markers',
    #marker_symbol='square',
    marker=dict(
        size=4,
        color=divergeData['color'],                # set color to an array/list of desired values
        #colorscale='Viridis',   # choose a colorscale
        opacity=1.0,
        line=dict(
                color='Black',
                width=2)
    )
)

fig = go.Figure(data = traces)

# tight layout
fig.update_layout(autosize=False,margin=dict(l=0, r=0, b=0, t=0))
fig.update_layout(scene = dict(
                    xaxis_title=r'Hx*cos(theta) (R)',
                    yaxis_title=r'Hx*sin(theta) (R)',
                    zaxis_title=r'Hy (R)'),
                    width=1000,height=1000,
                    margin=dict(r=20, b=10, l=10, t=10))
#fig.update_layout(showlegend=True,
#                  legend_title_text='Configurations\nRed=Diverge\nBlue=V-shape\nPink=Orbit\nOrange=Headbutt',
#                  font=dict(color="black",size=16))
camera = dict(
    eye=dict(x=0., y=0., z=2.5)
)

fig.update_layout(scene_camera=camera)
fig.show()

In [19]:
import plotly.graph_objects as go

#Grab Last timestep configuration data and combine in new dataset
#Using new dataset, plot in 3D the phase space (Hx, Hy, Theta)
#Maybe plotly, maybe matplotlib
#Plot Torus/Donut
#Make surface

allData = pd.read_csv(cwd_PYTHON+'/allData_Re10.csv',delimiter=' ')

endData = allData[allData['time'] == 20.0].copy()
endData = endData[endData['parHx'] >= 0.0].copy()
#endData = endData[endData['parHy'] <= 8.2].copy()
#endData = endData.sort_values(by=['parTheta','parHx','parHy'])
endData = endData.sort_values(by=['parHy','parTheta','parHx'])
endData['xval'] = 0.5*endData['parHx']*np.cos(endData['parTheta']*np.pi/180.0)
endData['yval'] = 0.5*endData['parHx']*np.sin(endData['parTheta']*np.pi/180.0)
endData['zval'] = 0.5*endData['parHy']
endData = endData.sort_values(by=['zval','yval','xval'])
endData = endData.reset_index(drop=True)
endData = WhichEndState(endData)

#Diverge = 0: Red
#V-Shape = 1: Blue
#Orbit = 2: pink
#NOTA = 3: black
    
#Create traces for each end state
traces = [0 for a in range(2)]
opaque = 0.2

#Orbit
orbitData = endData[endData['State'] == 2].copy()
orbitData = orbitData.reset_index(drop=True)

traces[0] = go.Mesh3d(x=orbitData['xval'], 
                     y=orbitData['yval'], 
                     z=orbitData['zval'], 
                     color=orbitData.loc[0,'color'], 
                     opacity=opaque,
                     #opacity=1.0,
                     alphahull=8
                     )

traces[1] = go.Scatter3d(
    x=orbitData['xval'],
    y=orbitData['yval'],
    z=orbitData['zval'],
    mode='markers',
    #marker_symbol='square',
    marker=dict(
        size=4,
        color=orbitData['color'],                # set color to an array/list of desired values
        #colorscale='Viridis',   # choose a colorscale
        opacity=1.0,
        line=dict(
                color='Black',
                width=2)
    )
)

fig = go.Figure(data = traces)

# tight layout
fig.update_layout(autosize=False,margin=dict(l=0, r=0, b=0, t=0))
fig.update_layout(scene = dict(
                    xaxis_title=r'Hx*cos(theta) (R)',
                    yaxis_title=r'Hx*sin(theta) (R)',
                    zaxis_title=r'Hy (R)'),
                    width=1000,height=1000,
                    margin=dict(r=20, b=10, l=10, t=10))
#fig.update_layout(showlegend=True,
#                  legend_title_text='Configurations\nRed=Diverge\nBlue=V-shape\nPink=Orbit\nOrange=Headbutt',
#                  font=dict(color="black",size=16))
camera = dict(
    eye=dict(x=0., y=0., z=2.5)
)

fig.update_layout(scene_camera=camera)
fig.show()

In [12]:
import plotly.graph_objects as go

#Grab Last timestep configuration data and combine in new dataset
#Using new dataset, plot in 3D the phase space (Hx, Hy, Theta)
#Maybe plotly, maybe matplotlib
#Plot Torus/Donut
#Make surface

allData = pd.read_csv(cwd_PYTHON+'/allData_Re10.csv',delimiter=' ')

endData = allData[allData['time'] == 20.0].copy()
endData = endData[endData['parHx'] >= 0.0].copy()
#endData = endData[endData['parHy'] <= 8.2].copy()
#endData = endData.sort_values(by=['parTheta','parHx','parHy'])
endData = endData.sort_values(by=['parHy','parTheta','parHx'])
endData['xval'] = 0.5*endData['parHx']*np.cos(endData['parTheta']*np.pi/180.0)
endData['yval'] = 0.5*endData['parHx']*np.sin(endData['parTheta']*np.pi/180.0)
endData['zval'] = 0.5*endData['parHy']
endData = endData.sort_values(by=['zval','yval','xval'])
endData = endData.reset_index(drop=True)
endData = WhichEndState(endData)

#Diverge = 0: Red
#V-Shape = 1: Blue
#Orbit = 2: pink
#NOTA = 3: black
    
#Create traces for each end state
traces = [0 for a in range(2)]
opaque = 0.2

#Divergence
headbuttData = endData[endData['State'] == 3].copy()
headbuttData = headbuttData.reset_index(drop=True)
traces[0] = go.Mesh3d(x=headbuttData['xval'], 
                     y=headbuttData['yval'], 
                     z=headbuttData['zval'], 
                     color=headbuttData.loc[0,'color'], 
                     opacity=opaque,
                     #opacity=1.0,
                     alphahull=5
                     )

traces[1] = go.Scatter3d(
    x=headbuttData['xval'],
    y=headbuttData['yval'],
    z=headbuttData['zval'],
    mode='markers',
    #marker_symbol='square',
    marker=dict(
        size=4,
        color=headbuttData['color'],                # set color to an array/list of desired values
        #colorscale='Viridis',   # choose a colorscale
        opacity=1.0,
        line=dict(
                color='Black',
                width=2)
    )
)

fig = go.Figure(data = traces)

# tight layout
fig.update_layout(autosize=False,margin=dict(l=0, r=0, b=0, t=0))
fig.update_layout(scene = dict(
                    xaxis_title=r'Hx*cos(theta) (R)',
                    yaxis_title=r'Hx*sin(theta) (R)',
                    zaxis_title=r'Hy (R)'),
                    width=1000,height=1000,
                    margin=dict(r=20, b=10, l=10, t=10))
#fig.update_layout(showlegend=True,
#                  legend_title_text='Configurations\nRed=Diverge\nBlue=V-shape\nPink=Orbit\nOrange=Headbutt',
#                  font=dict(color="black",size=16))
camera = dict(
    eye=dict(x=0., y=0., z=2.5)
)

fig.update_layout(scene_camera=camera)
fig.show()

In [15]:
import plotly.graph_objects as go

#Grab Last timestep configuration data and combine in new dataset
#Using new dataset, plot in 3D the phase space (Hx, Hy, Theta)
#Maybe plotly, maybe matplotlib
#Plot Torus/Donut
#Make surface

allData = pd.read_csv(cwd_PYTHON+'/allData_Re10.csv',delimiter=' ')

endData = allData[allData['time'] == 20.0].copy()
endData = endData[endData['parHx'] >= 0.0].copy()
#endData = endData[endData['parHy'] <= 8.2].copy()
#endData = endData.sort_values(by=['parTheta','parHx','parHy'])
endData = endData.sort_values(by=['parHy','parTheta','parHx'])
endData['xval'] = 0.5*endData['parHx']*np.cos(endData['parTheta']*np.pi/180.0)
endData['yval'] = 0.5*endData['parHx']*np.sin(endData['parTheta']*np.pi/180.0)
endData['zval'] = 0.5*endData['parHy']
endData = endData.sort_values(by=['zval','yval','xval'])
endData = endData.reset_index(drop=True)
endData = WhichEndState(endData)

#Diverge = 0: Red
#V-Shape = 1: Blue
#Orbit = 2: pink
#HeadButt = 3: orange
#NOTA = 4: black
    
#Create traces for each end state
traces = [0 for a in range(2)]
opaque = 0.2

#Divergence
NOTAData = endData[endData['State'] == 4].copy()
NOTAData = NOTAData.reset_index(drop=True)
traces[0] = go.Mesh3d(x=NOTAData['xval'], 
                     y=NOTAData['yval'], 
                     z=NOTAData['zval'], 
                     color=NOTAData.loc[0,'color'], 
                     opacity=opaque,
                     #opacity=1.0,
                     alphahull=5
                     )

traces[1] = go.Scatter3d(
    x=NOTAData['xval'],
    y=NOTAData['yval'],
    z=NOTAData['zval'],
    mode='markers',
    #marker_symbol='square',
    marker=dict(
        size=8,
        color=NOTAData['color'],                # set color to an array/list of desired values
        #colorscale='Viridis',   # choose a colorscale
        opacity=1.0,
        line=dict(
                color='Black',
                width=2)
    )
)

fig = go.Figure(data = traces[1])

# tight layout
fig.update_layout(autosize=False,margin=dict(l=0, r=0, b=0, t=0))
fig.update_layout(scene = dict(
                    xaxis_title=r'Hx*cos(theta) (R)',
                    yaxis_title=r'Hx*sin(theta) (R)',
                    zaxis_title=r'Hy (R)'),
                    width=1000,height=1000,
                    margin=dict(r=20, b=10, l=10, t=10))
#fig.update_layout(showlegend=True,
#                  legend_title_text='Configurations\nRed=Diverge\nBlue=V-shape\nPink=Orbit\nOrange=Headbutt',
#                  font=dict(color="black",size=16))
camera = dict(
    eye=dict(x=0., y=0., z=2.5)
)

fig.update_layout(scene_camera=camera)
fig.show()

In [7]:
import open3d as o3d

#Obtain Point cloud data
input_path=cwd_PYTHON+'/'
output_path=cwd_PYTHON+'/../Figures/3DTest/'
dataname="Vshape_pointcloud.csv"
point_cloud= np.loadtxt(input_path+dataname,skiprows=1)
#print(point_cloud[:10])

#Transform point cloud from Numpy to Open3D o3d.geometry.PointCloud()
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(point_cloud[:,:3])
pcd.colors = o3d.utility.Vector3dVector(point_cloud[:,3:6])
pcd.normals = o3d.utility.Vector3dVector(np.zeros((1,3))) #invalidate existing normals
pcd.estimate_normals()
#pcd.orient_normals_consistent_tangent_plane(k=7)
o3d.visualization.draw_geometries([pcd], point_show_normal=True)

#o3d.visualization.draw_geometries([pcd])

#Generate mesh using BPA method
distances = pcd.compute_nearest_neighbor_distance()
avg_dist = np.mean(distances)
radius = 3 * avg_dist

bpa_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_ball_pivoting(pcd,
                                                                o3d.utility.DoubleVector([radius, radius * 2]))

bpa_mesh.remove_degenerate_triangles()
bpa_mesh.remove_duplicated_triangles()
bpa_mesh.remove_duplicated_vertices()
bpa_mesh.remove_non_manifold_edges()
o3d.visualization.draw_geometries([pcd,bpa_mesh])

o3d.io.write_triangle_mesh(output_path+"bpa_mesh.ply", bpa_mesh)


'''
#Generate mesh using poisson method
poisson_mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=8, width=0, scale=1.1, linear_fit=False)[0]
bbox = pcd.get_axis_aligned_bounding_box()
p_mesh_crop = poisson_mesh.crop(bbox)
o3d.io.write_triangle_mesh(output_path+"p_mesh_c.ply", p_mesh_crop)
'''

sys.exit(0)

def lod_mesh_export(mesh, lods, extension, path):
    mesh_lods={}
    for i in lods:
        mesh_lod = mesh.simplify_quadric_decimation(i)
        o3d.io.write_triangle_mesh(path+"lod_"+str(i)+extension, mesh_lod)
        mesh_lods[i]=mesh_lod
    print("generation of "+str(i)+" LoD successful")
    return mesh_lods

my_lods = lod_mesh_export(bpa_mesh, [8000,800,300], ".ply", output_path)

o3d.visualization.draw_geometries([my_lods])


SystemExit: 0

/Users/thomas/anaconda3/envs/tf-research/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3351: UserWarning:

To exit: use 'exit', 'quit', or Ctrl-D.

